In [1]:
import sys,os
import pickle
import anndata
import pandas as pd
import numpy as np
import scanpy as sc
import diffxpy.api as de
import statsmodels.stats.multitest as multi
import pickle

In [2]:
adata = sc.read_h5ad(open("/nfs/team283/aa16/KR_NAS/data/CR_cells_AnnData.h5ad", "rb"))

In [3]:
cr_cell_nuclei = np.mean(adata.obs['nuclei'].loc[adata.obs['Pool'] == 'CRcells'])
cr_cell_reln = np.mean(adata[adata.obs['Pool'] == 'CRcells',np.array(adata.var == 'RELN')].X)

# Number of nuclei per AOI type:
print('CR cell ROIs mean nuclei:', np.mean(adata.obs['nuclei'].loc[adata.obs['Pool'] == 'CRcells']))
print('Residual ROIs mean nuclei:', np.mean(adata.obs['nuclei'].loc[adata.obs['Pool'] == 'Residual']))
# RELN expression per AOI type:
print('CR cell ROIs mean RELN expression:', np.mean(adata[adata.obs['Pool'] == 'CRcells', np.array(adata.var == 'RELN')].X))
print('Residual ROIs mean RELN expression:', np.mean(adata[adata.obs['Pool'] == 'Residual', np.array(adata.var == 'RELN')].X))
# NR2F1 expression in Frontal vs Occipital:
print('Frontal Cortex ROIs mean NR2F1 expression:', np.mean(adata[adata.obs['Tissue'] == 'Foetal brain (frontal lobe)', np.array(adata.var == 'NR2F1')].X))
print('Occipital Cortex ROIs mean NR2F1 expression:', np.mean(adata[adata.obs['Tissue'] == 'Foetal brain (occipital lobe)', np.array(adata.var == 'NR2F1')].X))

if np.mean(adata.obs['nuclei'].loc[adata.obs['Pool'] == 'CRcells']) != cr_cell_nuclei:
    raise ValueError()
elif np.mean(adata[adata.obs['Pool'] == 'CRcells',np.array(adata.var == 'RELN')].X) != cr_cell_reln:
    raise ValueError()
elif np.mean(adata[adata.obs['Tissue'] == 'Foetal brain (frontal lobe)', np.array(adata.var == 'NR2F1')].X) > np.mean(adata[adata.obs['Tissue'] == 'Foetal brain (occipital lobe)', np.array(adata.var == 'NR2F1')].X):
    raise ValueError()
else:
    print('Input data looks sensible.')

CR cell ROIs mean nuclei: 23.445945945945947
Residual ROIs mean nuclei: 136.55714285714285
CR cell ROIs mean RELN expression: 415.3784
Residual ROIs mean RELN expression: 13.985714
Frontal Cortex ROIs mean NR2F1 expression: 19.028572
Occipital Cortex ROIs mean NR2F1 expression: 47.048782
Input data looks sensible.


In [4]:
adata.var_names = adata.var['SYMBOL']

In [5]:
adata = adata[adata.obs['Pool'] == 'Residual',:]

In [6]:
if np.any(adata.obs['Pool'] != 'Residual'):
    raise ValueError()
elif np.sum(adata.obs['Pool'] == 'Residual') != len(adata.obs['Pool']):
     raise ValueError()
else:
    print('Input data still makes sense.')
    
adata.obs['Donor'] = [x.split('-')[0] for x in adata.obs['Sample slide ID']]
(adata.obs['Tissue'].astype(str) + ' ' + adata.obs['Pool'].astype(str)
 + ' ' + adata.obs['Donor'].astype(str)).value_counts()

Trying to set attribute `.obs` of view, copying.


Input data still makes sense.


Foetal brain (occipital lobe) Residual Hob7    12
Foetal brain (frontal lobe) Residual Hob8      12
Foetal brain (temporal lobe) Residual Hob7     10
Foetal brain (parietal lobe) Residual Hob8      9
Foetal brain (temporal lobe) Residual Hob8      9
Foetal brain (occipital lobe) Residual Hob8     8
Foetal brain (frontal lobe) Residual Hob7       5
Foetal brain (parietal lobe) Residual Hob7      5
dtype: int64

In [7]:
adata.X = adata.layers['X_corrected_naive_cpm']
res = {}
count = 0
tissues = np.unique(adata.obs['Tissue'])
for i in range(len(tissues)):
    for j in range((i+1),len(tissues)):
        print(i)
        print(j)
        if i !=j:
            new_key = str(tissues[i]) + '_vs_' + str(tissues[j])
            adata_subset = adata[[x in (tissues[i], tissues[j]) for x in adata.obs['Tissue']],:]
            print(new_key)
            print((adata_subset.obs['Tissue'].astype(str) + ' ' + adata_subset.obs['Pool'].astype(str)
 + ' ' + adata_subset.obs['Donor'].astype(str)).value_counts())
            res[new_key] = de.test.rank_test(
                data=adata_subset,
                grouping="Tissue",
                is_logged=False
            ).summary()
            
for i in range(len(res.keys())):
    
    print(list(res.keys())[i])
    
    tissue1 = list(res.keys())[i].split('_vs_')[0]
    tissue2 = list(res.keys())[i].split('_vs_')[1]
    print(list(res.keys())[i])
    print(tissue1)
    print(tissue2)
    
    if np.sum(np.array(res[list(res.keys())[i]]['gene']) == np.array(adata.var_names)) != len(adata.var_names):
        raise ValueError()
    
    res[list(res.keys())[i]]['log2_mean_' + tissue1] = np.array(np.log2(np.mean(adata[adata.obs['Tissue'] == tissue1,].X, axis = 0)))
    res[list(res.keys())[i]]['log2_mean_' + tissue2] = np.array(np.log2(np.mean(adata[adata.obs['Tissue'] == tissue2,].X, axis = 0)))
    res[list(res.keys())[i]]['log2_fold_change'] =  res[list(res.keys())[i]]['log2_mean_' + tissue1] - res[list(res.keys())[i]]['log2_mean_' + tissue2]
    
    subset1 = [adata.obs['Tissue'].iloc[j] == tissue1 and adata.obs['Donor'].iloc[j] == 'Hob7' for j in range(len(adata.obs['Donor']))]
    subset2 = [adata.obs['Tissue'].iloc[j] == tissue2 and adata.obs['Donor'].iloc[j] == 'Hob7' for j in range(len(adata.obs['Donor']))]
    res[list(res.keys())[i]]['log2_mean_Hob7_' + tissue1] = np.array(np.log2(np.mean(adata[subset1,].X, axis = 0)))
    res[list(res.keys())[i]]['log2_mean_Hob7_' + tissue2] = np.array(np.log2(np.mean(adata[subset2,].X, axis = 0)))
    res[list(res.keys())[i]]['log2_fold_change_Hob7'] =  res[list(res.keys())[i]]['log2_mean_Hob7_' + tissue1] - res[list(res.keys())[i]]['log2_mean_Hob7_' + tissue2]
    
    print((adata[subset1,].obs['Tissue'].astype(str) + ' ' + adata[subset1,].obs['Pool'].astype(str)
 + ' ' + adata[subset1,].obs['Donor'].astype(str)).value_counts())
    print((adata[subset2,].obs['Tissue'].astype(str) + ' ' + adata[subset2,].obs['Pool'].astype(str)
 + ' ' + adata[subset2,].obs['Donor'].astype(str)).value_counts())
    
    subset1 = [adata.obs['Tissue'].iloc[j] == tissue1 and adata.obs['Donor'].iloc[j] == 'Hob8' for j in range(len(adata.obs['Donor']))]
    subset2 = [adata.obs['Tissue'].iloc[j] == tissue2 and adata.obs['Donor'].iloc[j] == 'Hob8' for j in range(len(adata.obs['Donor']))]
    res[list(res.keys())[i]]['log2_mean_Hob8_' + tissue1] = np.array(np.log2(np.mean(adata[subset1,].X, axis = 0)))
    res[list(res.keys())[i]]['log2_mean_Hob8_' + tissue2] = np.array(np.log2(np.mean(adata[subset2,].X, axis = 0)))
    res[list(res.keys())[i]]['log2_fold_change_Hob8'] =  res[list(res.keys())[i]]['log2_mean_Hob8_' + tissue1] - res[list(res.keys())[i]]['log2_mean_Hob8_' + tissue2]
    
    print((adata[subset1,].obs['Tissue'].astype(str) + ' ' + adata[subset1,].obs['Pool'].astype(str)
 + ' ' + adata[subset1,].obs['Donor'].astype(str)).value_counts())
    print((adata[subset2,].obs['Tissue'].astype(str) + ' ' + adata[subset2,].obs['Pool'].astype(str)
 + ' ' + adata[subset2,].obs['Donor'].astype(str)).value_counts())
    
    res[list(res.keys())[i]] = res[list(res.keys())[i]].sort_values('pval')
    
res_naive = res

0
1
Foetal brain (frontal lobe)_vs_Foetal brain (occipital lobe)
Foetal brain (frontal lobe) Residual Hob8      12
Foetal brain (occipital lobe) Residual Hob7    12
Foetal brain (occipital lobe) Residual Hob8     8
Foetal brain (frontal lobe) Residual Hob7       5
dtype: int64
0
2
Foetal brain (frontal lobe)_vs_Foetal brain (parietal lobe)
Foetal brain (frontal lobe) Residual Hob8     12
Foetal brain (parietal lobe) Residual Hob8     9
Foetal brain (parietal lobe) Residual Hob7     5
Foetal brain (frontal lobe) Residual Hob7      5
dtype: int64
0
3
Foetal brain (frontal lobe)_vs_Foetal brain (temporal lobe)
Foetal brain (frontal lobe) Residual Hob8     12
Foetal brain (temporal lobe) Residual Hob7    10
Foetal brain (temporal lobe) Residual Hob8     9
Foetal brain (frontal lobe) Residual Hob7      5
dtype: int64
1
2
Foetal brain (occipital lobe)_vs_Foetal brain (parietal lobe)
Foetal brain (occipital lobe) Residual Hob7    12
Foetal brain (parietal lobe) Residual Hob8      9
Foetal bra

/nfs/team283/aa16/software/miniconda3/envs/countcorrect/lib/python3.7/site-packages/ipykernel_launcher.py:51: RuntimeWarning: divide by zero encountered in log2
/nfs/team283/aa16/software/miniconda3/envs/countcorrect/lib/python3.7/site-packages/ipykernel_launcher.py:52: RuntimeWarning: divide by zero encountered in log2
/nfs/team283/aa16/software/miniconda3/envs/countcorrect/lib/python3.7/site-packages/ipykernel_launcher.py:35: RuntimeWarning: divide by zero encountered in log2


Foetal brain (frontal lobe) Residual Hob8    12
dtype: int64
Foetal brain (occipital lobe) Residual Hob8    8
dtype: int64
Foetal brain (frontal lobe)_vs_Foetal brain (parietal lobe)
Foetal brain (frontal lobe)_vs_Foetal brain (parietal lobe)
Foetal brain (frontal lobe)
Foetal brain (parietal lobe)
Foetal brain (frontal lobe) Residual Hob7    5
dtype: int64


/nfs/team283/aa16/software/miniconda3/envs/countcorrect/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: divide by zero encountered in log2


Foetal brain (parietal lobe) Residual Hob7    5
dtype: int64
Foetal brain (frontal lobe) Residual Hob8    12
dtype: int64
Foetal brain (parietal lobe) Residual Hob8    9
dtype: int64
Foetal brain (frontal lobe)_vs_Foetal brain (temporal lobe)
Foetal brain (frontal lobe)_vs_Foetal brain (temporal lobe)
Foetal brain (frontal lobe)
Foetal brain (temporal lobe)
Foetal brain (frontal lobe) Residual Hob7    5
dtype: int64
Foetal brain (temporal lobe) Residual Hob7    10
dtype: int64
Foetal brain (frontal lobe) Residual Hob8    12
dtype: int64
Foetal brain (temporal lobe) Residual Hob8    9
dtype: int64
Foetal brain (occipital lobe)_vs_Foetal brain (parietal lobe)
Foetal brain (occipital lobe)_vs_Foetal brain (parietal lobe)
Foetal brain (occipital lobe)
Foetal brain (parietal lobe)
Foetal brain (occipital lobe) Residual Hob7    12
dtype: int64
Foetal brain (parietal lobe) Residual Hob7    5
dtype: int64
Foetal brain (occipital lobe) Residual Hob8    8
dtype: int64
Foetal brain (parietal lobe

/nfs/team283/aa16/software/miniconda3/envs/countcorrect/lib/python3.7/site-packages/ipykernel_launcher.py:34: RuntimeWarning: divide by zero encountered in log2
/nfs/team283/aa16/software/miniconda3/envs/countcorrect/lib/python3.7/site-packages/ipykernel_launcher.py:40: RuntimeWarning: divide by zero encountered in log2


Foetal brain (parietal lobe) Residual Hob8    9
dtype: int64
Foetal brain (temporal lobe) Residual Hob8    9
dtype: int64


In [8]:
pickle.dump(res_naive, open("/nfs/team283/aa16/KR_NAS/0_publication_tables/CR_cells_pairwiseDE_naive_residual.p", "wb" ))

In [9]:
# NR2F1 expression in Frontal vs Occipital:
print('Frontal Cortex ROIs mean NR2F1 expression:', np.mean(adata[adata.obs['Tissue'] == 'Foetal brain (frontal lobe)', np.array(adata.var == 'NR2F1')].X))
print('Occipital Cortex ROIs mean NR2F1 expression:', np.mean(adata[adata.obs['Tissue'] == 'Foetal brain (occipital lobe)', np.array(adata.var == 'NR2F1')].X))

if np.mean(adata.obs['nuclei'].loc[adata.obs['Pool'] == 'CRcells']) > np.mean(adata.obs['nuclei'].loc[adata.obs['Pool'] == 'Residual']):
    raise ValueError()
elif np.mean(adata[adata.obs['Pool'] == 'CRcells',np.array(adata.var == 'RELN')].X) < np.mean(adata[adata.obs['Pool'] == 'Residual',np.array(adata.var == 'RELN')].X):
    raise ValueError()
elif np.mean(adata[adata.obs['Tissue'] == 'Foetal brain (frontal lobe)', np.array(adata.var == 'NR2F1')].X) > np.mean(adata[adata.obs['Tissue'] == 'Foetal brain (occipital lobe)', np.array(adata.var == 'NR2F1')].X):
    raise ValueError()
else:
    print('Input data still looks sensible.')

Frontal Cortex ROIs mean NR2F1 expression: 211.7921053132102
Occipital Cortex ROIs mean NR2F1 expression: 523.3462340235658
Input data still looks sensible.


/nfs/team283/aa16/software/miniconda3/envs/countcorrect/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3438: RuntimeWarning: Mean of empty slice.
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/nfs/team283/aa16/software/miniconda3/envs/countcorrect/lib/python3.7/site-packages/numpy/core/_methods.py:182: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


In [10]:
adata.X = adata.layers['X_corrected_cc_cpm']
res = {}
count = 0
tissues = np.unique(adata.obs['Tissue'])
for i in range(len(tissues)):
    for j in range((i+1),len(tissues)):
        print(i)
        print(j)
        if i !=j:
            new_key = str(tissues[i]) + '_vs_' + str(tissues[j])
            adata_subset = adata[[x in (tissues[i], tissues[j]) for x in adata.obs['Tissue']],:]
            print(new_key)
            print((adata_subset.obs['Tissue'].astype(str) + ' ' + adata_subset.obs['Pool'].astype(str)
 + ' ' + adata_subset.obs['Donor'].astype(str)).value_counts())
            res[new_key] = de.test.rank_test(
                data=adata_subset,
                grouping="Tissue",
                is_logged=False
            ).summary()
            
for i in range(len(res.keys())):
    
    print(list(res.keys())[i])
    
    tissue1 = list(res.keys())[i].split('_vs_')[0]
    tissue2 = list(res.keys())[i].split('_vs_')[1]
    print(list(res.keys())[i])
    print(tissue1)
    print(tissue2)
    
    res[list(res.keys())[i]]['log2_mean_' + tissue1] = np.array(np.log2(np.mean(adata[adata.obs['Tissue'] == tissue1,].X, axis = 0)))
    res[list(res.keys())[i]]['log2_mean_' + tissue2] = np.array(np.log2(np.mean(adata[adata.obs['Tissue'] == tissue2,].X, axis = 0)))
    res[list(res.keys())[i]]['log2_fold_change'] =  res[list(res.keys())[i]]['log2_mean_' + tissue1] - res[list(res.keys())[i]]['log2_mean_' + tissue2]
    
    if np.sum(np.array(res[list(res.keys())[i]]['gene']) == np.array(adata.var_names)) != len(adata.var_names):
        raise ValueError()
    
    subset1 = [adata.obs['Tissue'].iloc[j] == tissue1 and adata.obs['Donor'].iloc[j] == 'Hob7' for j in range(len(adata.obs['Donor']))]
    subset2 = [adata.obs['Tissue'].iloc[j] == tissue2 and adata.obs['Donor'].iloc[j] == 'Hob7' for j in range(len(adata.obs['Donor']))]
    res[list(res.keys())[i]]['log2_mean_Hob7_' + tissue1] = np.array(np.log2(np.mean(adata[subset1,].X, axis = 0)))
    res[list(res.keys())[i]]['log2_mean_Hob7_' + tissue2] = np.array(np.log2(np.mean(adata[subset2,].X, axis = 0)))
    res[list(res.keys())[i]]['log2_fold_change_Hob7'] =  res[list(res.keys())[i]]['log2_mean_Hob7_' + tissue1] - res[list(res.keys())[i]]['log2_mean_Hob7_' + tissue2]
    
    print((adata[subset1,].obs['Tissue'].astype(str) + ' ' + adata[subset1,].obs['Pool'].astype(str)
 + ' ' + adata[subset1,].obs['Donor'].astype(str)).value_counts())
    print((adata[subset2,].obs['Tissue'].astype(str) + ' ' + adata[subset2,].obs['Pool'].astype(str)
 + ' ' + adata[subset2,].obs['Donor'].astype(str)).value_counts())
    
    subset1 = [adata.obs['Tissue'].iloc[j] == tissue1 and adata.obs['Donor'].iloc[j] == 'Hob8' for j in range(len(adata.obs['Donor']))]
    subset2 = [adata.obs['Tissue'].iloc[j] == tissue2 and adata.obs['Donor'].iloc[j] == 'Hob8' for j in range(len(adata.obs['Donor']))]
    res[list(res.keys())[i]]['log2_mean_Hob8_' + tissue1] = np.array(np.log2(np.mean(adata[subset1,].X, axis = 0)))
    res[list(res.keys())[i]]['log2_mean_Hob8_' + tissue2] = np.array(np.log2(np.mean(adata[subset2,].X, axis = 0)))
    res[list(res.keys())[i]]['log2_fold_change_Hob8'] =  res[list(res.keys())[i]]['log2_mean_Hob8_' + tissue1] - res[list(res.keys())[i]]['log2_mean_Hob8_' + tissue2]
    
    print((adata[subset1,].obs['Tissue'].astype(str) + ' ' + adata[subset1,].obs['Pool'].astype(str)
 + ' ' + adata[subset1,].obs['Donor'].astype(str)).value_counts())
    print((adata[subset2,].obs['Tissue'].astype(str) + ' ' + adata[subset2,].obs['Pool'].astype(str)
 + ' ' + adata[subset2,].obs['Donor'].astype(str)).value_counts())
    
    if np.sum(np.array(res[list(res.keys())[i]]['gene']) == np.array(adata.var_names)) != len(adata.var_names):
        raise ValueError()
    
    res[list(res.keys())[i]] = res[list(res.keys())[i]].sort_values('pval')
    
res_cc = res

0
1
Foetal brain (frontal lobe)_vs_Foetal brain (occipital lobe)
Foetal brain (frontal lobe) Residual Hob8      12
Foetal brain (occipital lobe) Residual Hob7    12
Foetal brain (occipital lobe) Residual Hob8     8
Foetal brain (frontal lobe) Residual Hob7       5
dtype: int64
0
2
Foetal brain (frontal lobe)_vs_Foetal brain (parietal lobe)
Foetal brain (frontal lobe) Residual Hob8     12
Foetal brain (parietal lobe) Residual Hob8     9
Foetal brain (parietal lobe) Residual Hob7     5
Foetal brain (frontal lobe) Residual Hob7      5
dtype: int64
0
3
Foetal brain (frontal lobe)_vs_Foetal brain (temporal lobe)
Foetal brain (frontal lobe) Residual Hob8     12
Foetal brain (temporal lobe) Residual Hob7    10
Foetal brain (temporal lobe) Residual Hob8     9
Foetal brain (frontal lobe) Residual Hob7      5
dtype: int64
1
2
Foetal brain (occipital lobe)_vs_Foetal brain (parietal lobe)
Foetal brain (occipital lobe) Residual Hob7    12
Foetal brain (parietal lobe) Residual Hob8      9
Foetal bra

/nfs/team283/aa16/software/miniconda3/envs/countcorrect/lib/python3.7/site-packages/ipykernel_launcher.py:40: RuntimeWarning: divide by zero encountered in log2
/nfs/team283/aa16/software/miniconda3/envs/countcorrect/lib/python3.7/site-packages/ipykernel_launcher.py:51: RuntimeWarning: divide by zero encountered in log2
/nfs/team283/aa16/software/miniconda3/envs/countcorrect/lib/python3.7/site-packages/ipykernel_launcher.py:52: RuntimeWarning: divide by zero encountered in log2


Foetal brain (frontal lobe) Residual Hob8    12
dtype: int64
Foetal brain (occipital lobe) Residual Hob8    8
dtype: int64
Foetal brain (frontal lobe)_vs_Foetal brain (parietal lobe)
Foetal brain (frontal lobe)_vs_Foetal brain (parietal lobe)
Foetal brain (frontal lobe)
Foetal brain (parietal lobe)
Foetal brain (frontal lobe) Residual Hob7    5
dtype: int64
Foetal brain (parietal lobe) Residual Hob7    5
dtype: int64


/nfs/team283/aa16/software/miniconda3/envs/countcorrect/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: divide by zero encountered in log2


Foetal brain (frontal lobe) Residual Hob8    12
dtype: int64
Foetal brain (parietal lobe) Residual Hob8    9
dtype: int64
Foetal brain (frontal lobe)_vs_Foetal brain (temporal lobe)
Foetal brain (frontal lobe)_vs_Foetal brain (temporal lobe)
Foetal brain (frontal lobe)
Foetal brain (temporal lobe)
Foetal brain (frontal lobe) Residual Hob7    5
dtype: int64
Foetal brain (temporal lobe) Residual Hob7    10
dtype: int64
Foetal brain (frontal lobe) Residual Hob8    12
dtype: int64
Foetal brain (temporal lobe) Residual Hob8    9
dtype: int64
Foetal brain (occipital lobe)_vs_Foetal brain (parietal lobe)
Foetal brain (occipital lobe)_vs_Foetal brain (parietal lobe)
Foetal brain (occipital lobe)
Foetal brain (parietal lobe)
Foetal brain (occipital lobe) Residual Hob7    12
dtype: int64
Foetal brain (parietal lobe) Residual Hob7    5
dtype: int64
Foetal brain (occipital lobe) Residual Hob8    8
dtype: int64
Foetal brain (parietal lobe) Residual Hob8    9
dtype: int64
Foetal brain (occipital lob

In [11]:
pickle.dump(res_cc, open("/nfs/team283/aa16/KR_NAS/0_publication_tables/CR_cells_pairwiseDE_cc_residual.p", "wb" ))

Save all data on pairwise differential expression in CR cells in one data frame:

In [12]:
allResults = []

include_sc_array = [True, True, False, True, False, False]
key_wta_array = ['Foetal brain (frontal lobe)_vs_Foetal brain (occipital lobe)',
                'Foetal brain (frontal lobe)_vs_Foetal brain (parietal lobe)',
                'Foetal brain (frontal lobe)_vs_Foetal brain (temporal lobe)',
                'Foetal brain (occipital lobe)_vs_Foetal brain (parietal lobe)',
                'Foetal brain (occipital lobe)_vs_Foetal brain (temporal lobe)',
                'Foetal brain (parietal lobe)_vs_Foetal brain (temporal lobe)']
key_sc_array = ['PFC_vs_V1', 'PFC_vs_parietal', None, 'V1_vs_parietal', None, None]

for j in range(len(key_wta_array)):
    
    print(j)

    res_sc = pickle.load(open("/nfs/team283/brainData/human_fetal/Bhaduri2021/CR_cells_pairwiseDE.p", "rb" ))
    res_cc = pickle.load(open("/nfs/team283/aa16/KR_NAS/0_publication_tables/CR_cells_pairwiseDE_cc_residual.p", "rb" ))
    res_naive = pickle.load(open("/nfs/team283/aa16/KR_NAS/0_publication_tables/CR_cells_pairwiseDE_naive_residual.p", "rb" ))
    
    tissue1_wta = key_wta_array[j].split('_vs_')[0]
    tissue2_wta = key_wta_array[j].split('_vs_')[1]
    
    include_sc = include_sc_array[j]
    
    if include_sc:
        tissue1_sc = key_sc_array[j].split('_vs_')[0]
        tissue2_sc = key_sc_array[j].split('_vs_')[1]

    key_wta = key_wta_array[j]
    key_sc = key_sc_array[j]

    if include_sc:
        res_sc = res_sc[key_sc]
    res_cc = res_cc[key_wta]
    res_naive = res_naive[key_wta]
    res_tidy = pd.DataFrame(index = list(range(len(res_cc['gene']))), columns = ('gene', 'qval_cc', 'qval_naive', 'qval_sc',
                                       'log2_mean_wta' + tissue1_wta, 'log2_mean_wta' + tissue2_wta,
                                       'log2_mean_sc' + tissue1_sc, 'log2_mean_sc' + tissue2_sc,                                         
                                       'log2fc_cc', 'log2fc_naive', 'log2fc_sc',
                                       'log2fc_Hob7_cc', 'log2fc_Hob8_cc',
                                       'log2fc_Hob7_naive', 'log2fc_Hob8_naive'))
    
    print(key_wta)
    print(key_sc)
    print(tissue1_wta)
    print(tissue2_wta)
    
    for i in range(len(res_cc['gene'])):
        res_tidy['gene'].iloc[i]  = res_cc['gene'].iloc[i]
        
        index_naive = res_naive['gene'] == res_cc['gene'].iloc[i]
        if include_sc:
            index_sc = res_sc['gene'] == res_cc['gene'].iloc[i]
        else:
            index_sc = (False, False, False)
        
        if res_naive['gene'].loc[index_naive].iloc[0] != res_cc['gene'].iloc[i]:
            print(res_naive['gene'].loc[index_naive])
            print(res_cc['gene'].iloc[i])
            raise ValueError()
        
        res_tidy['qval_cc'].iloc[i] = res_cc['qval'].iloc[i]
        res_tidy['qval_naive'].iloc[i]  = res_naive['qval'].loc[index_naive].iloc[0]
        if np.sum(index_sc):
            res_tidy['qval_sc'].iloc[i]  = res_sc['qval'].loc[index_sc].iloc[0]
        
        res_tidy['log2_mean_wta' + tissue1_wta].iloc[i] = res_cc['log2_mean_' + tissue1_wta].iloc[i]
        res_tidy['log2_mean_wta' + tissue2_wta].iloc[i] = res_cc['log2_mean_' + tissue2_wta].iloc[i]
        
        if np.sum(index_sc):
            res_tidy['log2_mean_sc' + tissue1_sc].iloc[i] = res_sc['log2_mean_' + tissue1_sc].loc[index_sc].iloc[0]
            res_tidy['log2_mean_sc' + tissue2_sc].iloc[i] = res_sc['log2_mean_' + tissue2_sc].loc[index_sc].iloc[0]
        
        res_tidy['log2fc_cc'].iloc[i]  = res_cc['log2_fold_change'].iloc[i]
        res_tidy['log2fc_naive'].iloc[i]  = res_naive['log2_fold_change'].loc[index_naive].iloc[0]
        if np.sum(index_sc):
            res_tidy['log2fc_sc'].iloc[i]  = res_sc['log2_fold_change'].loc[index_sc].iloc[0]

        res_tidy['log2fc_Hob7_cc'].iloc[i]  = res_cc['log2_fold_change_Hob7'].iloc[i]
        res_tidy['log2fc_Hob7_naive'].iloc[i]  = res_naive['log2_fold_change_Hob7'].loc[index_naive].iloc[0]

        res_tidy['log2fc_Hob8_cc'].iloc[i]  = res_cc['log2_fold_change_Hob8'].iloc[i]
        res_tidy['log2fc_Hob8_naive'].iloc[i]  = res_naive['log2_fold_change_Hob8'].loc[index_naive].iloc[0]
        
        if res_tidy['log2fc_cc'].iloc[i] != (res_tidy['log2_mean_wta' + tissue1_wta].iloc[i] - res_tidy['log2_mean_wta' + tissue2_wta].iloc[i]):
            raise ValueError()
            
    allResults += [res_tidy]

0
Foetal brain (frontal lobe)_vs_Foetal brain (occipital lobe)
PFC_vs_V1
Foetal brain (frontal lobe)
Foetal brain (occipital lobe)
1
Foetal brain (frontal lobe)_vs_Foetal brain (parietal lobe)
PFC_vs_parietal
Foetal brain (frontal lobe)
Foetal brain (parietal lobe)
2
Foetal brain (frontal lobe)_vs_Foetal brain (temporal lobe)
None
Foetal brain (frontal lobe)
Foetal brain (temporal lobe)
3
Foetal brain (occipital lobe)_vs_Foetal brain (parietal lobe)
V1_vs_parietal
Foetal brain (occipital lobe)
Foetal brain (parietal lobe)
4
Foetal brain (occipital lobe)_vs_Foetal brain (temporal lobe)
None
Foetal brain (occipital lobe)
Foetal brain (temporal lobe)
5
Foetal brain (parietal lobe)_vs_Foetal brain (temporal lobe)
None
Foetal brain (parietal lobe)
Foetal brain (temporal lobe)


In [13]:
pickle.dump(allResults, open("/nfs/team283/aa16/KR_NAS/0_publication_tables/CR_cells_pairwiseDE_allResults_residual.p", "wb" ))

In [14]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('/nfs/team283/aa16/KR_NAS/0_publication_tables/CR_cell_DE_results_residual.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
for i in range(len(allResults)):
    allResults[i].to_excel(writer, sheet_name=key_wta_array[i].split('Foetal brain')[1].split('lobe')[0].split('(')[1]
                           + 'vs '
                           + key_wta_array[i].split('Foetal brain')[2].split('lobe')[0].split('(')[1])

# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [15]:
res_sc = pickle.load(open("/nfs/team283/brainData/human_fetal/Bhaduri2021/CR_cells_pairwiseDE.p", "rb" ))
res_cc = pickle.load(open("/nfs/team283/aa16/KR_NAS/0_publication_tables/CR_cells_pairwiseDE_cc_residual.p", "rb" ))
res_naive = pickle.load(open("/nfs/team283/aa16/KR_NAS/0_publication_tables/CR_cells_pairwiseDE_naive_residual.p", "rb" ))

In [16]:
res_naive[list(res_naive.keys())[0]]

,gene,pval,qval,log2fc,mean,zero_mean,zero_variance,log2_mean_Foetal brain (frontal lobe),log2_mean_Foetal brain (occipital lobe),log2_fold_change,log2_mean_Hob7_Foetal brain (frontal lobe),log2_mean_Hob7_Foetal brain (occipital lobe),log2_fold_change_Hob7,log2_mean_Hob8_Foetal brain (frontal lobe),log2_mean_Hob8_Foetal brain (occipital lobe),log2_fold_change_Hob8
10607,NTNG2,0.000002,0.012018,3.145700,111.573020,False,False,4.411364,7.557064,-3.145700,4.830730,7.212134,-2.381404,4.192837,7.956488,-3.763652
17917,ZBTB18,0.000003,0.012018,2.029356,247.925859,False,False,6.539051,8.568407,-2.029356,5.844225,8.586250,-2.742025,6.752252,8.541222,-1.788970
5949,GAS2L3,0.000003,0.012018,1.556226,146.464876,False,False,6.159420,7.715646,-1.556226,6.095373,7.391732,-1.296359,6.185290,8.095989,-1.910699
747,ANP32B,0.000004,0.012018,1.895587,137.756236,False,False,5.801074,7.696660,-1.895587,6.579756,7.469440,-0.889685,5.290306,7.981862,-2.691556
150,ACKR3,0.000004,0.012018,-2.112652,63.448324,False,False,6.762373,4.649721,2.112652,6.799842,3.737625,3.062217,6.746469,5.417699,1.328770
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17308,UGT2A1,1.000000,1.000000,-0.058346,15.621478,False,False,3.996704,3.938358,0.058346,4.709557,4.029172,0.680384,3.550030,3.790427,-0.240396
17354,UNCX,1.000000,1.000000,-0.121789,13.433352,False,False,3.812300,3.690511,0.121789,3.306645,3.292944,0.013700,3.979904,4.135489,-0.155585
1836,C1orf210,1.000000,1.000000,0.052861,6.464702,False,False,2.663770,2.716631,-0.052861,3.306645,3.241885,0.064759,2.279314,1.165402,1.113912
12324,PPP1R2C,1.000000,1.000000,0.031098,19.837643,False,False,4.293276,4.324374,-0.031098,5.385833,4.157149,1.228684,3.372208,4.543674,-1.171466


In [17]:
res_cc[list(res_cc.keys())[0]]

,gene,pval,qval,log2fc,mean,zero_mean,zero_variance,log2_mean_Foetal brain (frontal lobe),log2_mean_Foetal brain (occipital lobe),log2_fold_change,log2_mean_Hob7_Foetal brain (frontal lobe),log2_mean_Hob7_Foetal brain (occipital lobe),log2_fold_change_Hob7,log2_mean_Hob8_Foetal brain (frontal lobe),log2_mean_Hob8_Foetal brain (occipital lobe),log2_fold_change_Hob8
10607,NTNG2,7.287518e-07,0.013611,2.910784,119.048668,False,False,4.717659,7.628443,-2.910784,4.372009,7.348516,-2.976507,4.840358,7.967068,-3.126710
18015,ZEB1,2.895720e-06,0.015680,1.526393,309.639082,False,False,7.262543,8.788936,-1.526393,6.756474,8.653276,-1.896802,7.430255,8.971145,-1.540890
15389,SST,3.340219e-06,0.015680,-1.564387,143.825510,False,False,7.807019,6.242632,1.564387,7.720316,6.166502,1.553814,7.841661,6.349786,1.491875
17917,ZBTB18,3.358112e-06,0.015680,1.796772,275.694917,False,False,6.881948,8.678720,-1.796772,6.484665,8.703451,-2.218786,7.019841,8.640808,-1.620968
5949,GAS2L3,5.593286e-06,0.018519,1.453775,143.918966,False,False,6.212960,7.666735,-1.453775,5.967937,7.369328,-1.401391,6.303925,8.022236,-1.718311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9758,MTMR4,1.000000e+00,1.000000,0.043113,55.169462,False,False,5.762334,5.805447,-0.043113,5.399162,5.759085,-0.359923,5.890268,5.872311,0.017957
3599,CRYL1,1.000000e+00,1.000000,0.281718,23.211190,False,False,4.377684,4.659402,-0.281718,4.868447,4.632994,0.235454,4.110902,4.698129,-0.587227
3594,CRYGB,1.000000e+00,1.000000,-0.127108,21.048257,False,False,4.462942,4.335834,0.127108,4.896469,4.204868,0.691600,4.235159,4.512374,-0.277216
3722,CTSS,1.000000e+00,1.000000,0.391839,20.725855,False,False,4.148475,4.540314,-0.391839,5.214807,4.057131,1.157676,3.270484,5.053202,-1.782718


Now make a little summary of results for standard areal markers:

In [2]:
frontal_markers = ['CLMP', 'DIAHPH3', 'CTNNBL1', 'CBLN2', 'VSTM2L',
                   'CPNE8', 'SLN', 'CNTN4', 'MB21D2']
occipital_markers = ['PENK', 'EPHA3', 'WSCD1', 'NPY', 'TENM4', 'NR2F1',
                     'FGFR3', 'LHX2', 'WNT7B']

In [10]:
res_cr = pickle.load(open("/nfs/team283/aa16/KR_NAS/0_publication_tables/CR_cells_pairwiseDE_cc.p", "rb" ))
res_residual = pickle.load(open("/nfs/team283/aa16/KR_NAS/0_publication_tables/CR_cells_pairwiseDE_cc_residual.p", "rb" ))
res_cr = res_cr['Foetal brain (frontal lobe)_vs_Foetal brain (occipital lobe)']
res_residual = res_residual['Foetal brain (frontal lobe)_vs_Foetal brain (occipital lobe)']

In [34]:
tab = pd.DataFrame(index = [f for f in np.array(res_cr['gene']) if f in frontal_markers],
                   columns = ['qval CRcells', 'qval Residuals',
                                                'log2_mean frontal_lobe CRcells', 'log2_mean frontal_lobe Residuals',
                                                 'log2_mean occipital_lobe CRcells', 'log2_mean occipital_lobe Residuals',
                                                 'log2_fold_change CRcells', 'log2_fold_change Residuals'])

tab['log2_mean frontal_lobe CRcells'] = np.array(res_cr.loc[[g in frontal_markers for g in res_cr['gene']],'log2_mean_Foetal brain (frontal lobe)'])
tab['log2_mean occipital_lobe CRcells'] = np.array(res_cr.loc[[g in frontal_markers for g in res_cr['gene']],'log2_mean_Foetal brain (occipital lobe)'])
tab['log2_fold_change CRcells'] = np.array(res_cr.loc[[g in frontal_markers for g in res_cr['gene']],'log2_fold_change'])
tab['qval CRcells'] = np.array(res_cr.loc[[g in frontal_markers for g in res_cr['gene']],'qval'])

tab['log2_mean frontal_lobe Residuals'] = np.array(res_residual.loc[[g in frontal_markers for g in res_residual['gene']],'log2_mean_Foetal brain (frontal lobe)'])
tab['log2_mean occipital_lobe Residuals'] = np.array(res_residual.loc[[g in frontal_markers for g in res_residual['gene']],'log2_mean_Foetal brain (occipital lobe)'])
tab['log2_fold_change Residuals'] = np.array(res_residual.loc[[g in frontal_markers for g in res_residual['gene']],'log2_fold_change'])
tab['qval Residuals'] = np.array(res_residual.loc[[g in frontal_markers for g in res_residual['gene']],'qval'])

In [35]:
tab

,qval CRcells,qval Residuals,log2_mean frontal_lobe CRcells,log2_mean frontal_lobe Residuals,log2_mean occipital_lobe CRcells,log2_mean occipital_lobe Residuals,log2_fold_change CRcells,log2_fold_change Residuals
VSTM2L,0.204147,0.081996,6.875866,6.708058,5.734778,5.201513,1.141088,1.506544
CBLN2,0.370649,0.254667,5.980787,7.797857,3.300990,6.017461,2.679797,1.780396
CPNE8,0.648612,0.262554,3.604291,5.468096,3.993900,3.753278,-0.389609,1.714817
CNTN4,0.770061,0.873119,6.253077,4.928391,6.524736,4.210004,-0.271659,0.718387
CTNNBL1,0.877767,0.873157,5.506515,5.639846,5.447600,5.324627,0.058915,0.315219
MB21D2,0.902599,0.966303,4.712270,4.236908,3.077444,3.897795,1.634827,0.339113
CLMP,0.922679,0.971941,4.598465,5.852460,1.355273,6.309664,3.243192,-0.457204
SLN,0.924558,0.990947,4.856048,4.311942,3.267085,4.776789,1.588962,-0.464847


In [36]:
tab = pd.DataFrame(index = [f for f in np.array(res_cr['gene']) if f in occipital_markers],
                   columns = ['qval CRcells', 'qval Residuals',
                                                'log2_mean frontal_lobe CRcells', 'log2_mean frontal_lobe Residuals',
                                                 'log2_mean occipital_lobe CRcells', 'log2_mean occipital_lobe Residuals',
                                                 'log2_fold_change CRcells', 'log2_fold_change Residuals'])

tab['log2_mean frontal_lobe CRcells'] = np.array(res_cr.loc[[g in occipital_markers for g in res_cr['gene']],'log2_mean_Foetal brain (frontal lobe)'])
tab['log2_mean occipital_lobe CRcells'] = np.array(res_cr.loc[[g in occipital_markers for g in res_cr['gene']],'log2_mean_Foetal brain (occipital lobe)'])
tab['log2_fold_change CRcells'] = np.array(res_cr.loc[[g in occipital_markers for g in res_cr['gene']],'log2_fold_change'])
tab['qval CRcells'] = np.array(res_cr.loc[[g in occipital_markers for g in res_cr['gene']],'qval'])

tab['log2_mean frontal_lobe Residuals'] = np.array(res_residual.loc[[g in occipital_markers for g in res_residual['gene']],'log2_mean_Foetal brain (frontal lobe)'])
tab['log2_mean occipital_lobe Residuals'] = np.array(res_residual.loc[[g in occipital_markers for g in res_residual['gene']],'log2_mean_Foetal brain (occipital lobe)'])
tab['log2_fold_change Residuals'] = np.array(res_residual.loc[[g in occipital_markers for g in res_residual['gene']],'log2_fold_change'])
tab['qval Residuals'] = np.array(res_residual.loc[[g in occipital_markers for g in res_residual['gene']],'qval'])

In [37]:
tab

,qval CRcells,qval Residuals,log2_mean frontal_lobe CRcells,log2_mean frontal_lobe Residuals,log2_mean occipital_lobe CRcells,log2_mean occipital_lobe Residuals,log2_fold_change CRcells,log2_fold_change Residuals
NR2F1,0.043009,0.023118,7.076248,7.762030,8.351682,8.987090,-1.275433,-1.225060
WNT7B,0.154058,0.714990,5.223329,5.419618,6.672740,6.420948,-1.449411,-1.001330
TENM4,0.374228,0.778147,6.000091,6.234596,6.851875,6.831686,-0.851783,-0.597090
LHX2,0.541438,0.816174,5.763169,4.204345,6.344384,4.673646,-0.581215,-0.469302
NPY,0.671176,0.958867,2.996753,4.938436,4.031199,4.225268,-1.034446,0.713168
FGFR3,0.711076,0.971941,4.797007,3.969427,3.717934,3.636176,1.079072,0.333252
EPHA3,0.934310,0.971941,6.678892,5.397189,6.628505,5.572106,0.050387,-0.174917
WSCD1,0.943848,0.997750,4.612375,4.666704,3.320950,4.662734,1.291425,0.003970
PENK,0.949785,1.000000,4.102215,5.905521,2.311502,5.781213,1.790713,0.124308
